# Manual human labeling

In [1]:
import polars as pl
import os
pl.Config.set_fmt_str_lengths(400)
import json

In [2]:
DATA_PATH = os.getenv("DATA_PATH", "")

In [3]:
# Load fnames for later manual inspection
fnames = pl.read_parquet(f"{DATA_PATH}/raw/farcaster-fnames-0-1730134800.parquet")
last_fnames = fnames[["fid","updated_at"]].group_by("fid").max()
last_fnames = last_fnames.join(fnames,on=["fid","updated_at"],how="left",coalesce=True)[["fid","fname"]]
last_fnames

fid,fname
i64,str
473201,"""cheninator"""
7080,"""adamch"""
512823,"""glaine25"""
272148,"""gsus1"""
864229,"""vantoan10"""
…,…
269060,"""0x213"""
721069,"""thedrsurendra"""
272868,"""krauser"""


### 1. Random samples

In [4]:
pl.read_parquet(f"{DATA_PATH}/raw/farcaster-casts-0-1730134800.parquet",n_rows=1)

created_at,updated_at,deleted_at,timestamp,fid,hash,parent_hash,parent_fid,parent_url,text,embeds,mentions,mentions_positions,root_parent_hash,root_parent_url,id
datetime[ns],datetime[ns],datetime[ns],datetime[ns],i64,binary,binary,i64,str,str,str,str,str,binary,str,i64
2023-09-05 23:07:28.064999,2023-09-05 23:07:28.064999,null,2022-12-20 20:15:00,1,"b""\xfc\x10\xf4\x0b\x09\x87\xdbr\xb9\x18\xbd9\xcc\xbe6s\xe1E\xfbg""","b""\xd7l\xe4\xaa|<Z\\x0e\x19\xe9\xfeE1\x90\xb97t\x88\xd7""",3,null,"""testing""","""[]""","""[]""","""[]""","b""\xd7l\xe4\xaa|<Z\\x0e\x19\xe9\xfeE1\x90\xb97t\x88\xd7""",null,4


In [5]:
# Get fids and the last time they have casted
casts_lazy = pl.scan_parquet(f"{DATA_PATH}/raw/farcaster-casts-0-1730134800.parquet")
casts_lazy = casts_lazy.select(["fid", "created_at"])
casts_lazy = casts_lazy.sort("created_at").group_by("fid").last()
casts = casts_lazy.collect()
casts

fid,created_at
i64,datetime[ns]
11290,2024-03-22 14:05:43.316566
288154,2024-10-11 09:47:42.074322
241482,2024-06-05 02:09:46.657673
431543,2024-07-08 10:46:21.127428
744414,2024-10-13 04:18:55.309220
…,…
806392,2024-10-16 16:54:59.879006
366099,2024-10-05 14:30:55.130735
283494,2024-02-08 01:08:41.278402


In [6]:
# Filter by users that casted at least one time in the last month
max_datetime = casts.select(pl.col("created_at").max().dt.offset_by("-1mo"))
casts.filter(pl.col("created_at") > max_datetime )

fid,created_at
i64,datetime[ns]
288154,2024-10-11 09:47:42.074322
744414,2024-10-13 04:18:55.309220
16920,2024-10-16 03:45:37.642763
657766,2024-10-28 16:01:53.363036
285495,2024-10-23 11:41:49.167612
…,…
769938,2024-10-05 04:03:31.012983
806392,2024-10-16 16:54:59.879006
366099,2024-10-05 14:30:55.130735


In [7]:
df = casts.join(last_fnames,how="left",on="fid",coalesce=True)[["fid","fname"]]
df = df.sample(len(df)-1,seed=1)
df

fid,fname
i64,str
553048,"""wangzaimiaomiao"""
575709,"""kwasedoo"""
579062,"""dexter88"""
780050,"""jnsource"""
668542,"""pillorebistse"""
…,…
498233,"""thais-stein"""
269574,"""jucikdt"""
675040,"""jiuzhi"""


In [8]:
# random_samples = {str(value): True for value in df[0:1000]['fname']}
# random_samples

In [9]:
random_samples = {
  'shenxinlin': True,
 'mandana': True,
 'zeta-tauri.eth': True,
 'lebeda': True,
 'hadleyla': True,
 'darkninja': True,
 'svuille': True,
 'fhftyr': True,
 'dablaze01': True,
 'armis': True,
 'kraftgas': True,
 'epicoter': True,
 'funkyfred': True,
 'jedwill': True,
 'kharkivskiiiii': True,
 'famouskonrad': True,
 'sundii': True,
 'smhsince91': True,
 'lukata': True,
 'worshipmusic': True,
 'jayden-nft': True,
 'franklin': False,
 'rnozofoqa': True,
 'dogerfroger': True,
 'phamhao0315': True,
 'buttercookie': True,
 'nidiff6': True,
 'jacc-blacc': True,
 'kojeck': True,
 'drptphungct': True,
 'giveme15mins': True,
 'iamgaruda': True,
 'ibiber': True,
 'ubkings': True,
 'annetgrib': True,
 'cryptoleononx': True,
 'mirain': True
 }

In [10]:
list_of_humans1 = [key for key,value in random_samples.items() if value is False]
list_of_humans1

['franklin']

With random samples, there is a very high number of bots, so it was not feasible to perform a manual labeling. Lets take a look at (a) ENS verified accounts and (b) users that didn't cast bot words.

### 2. Verified ENS accounts

In [11]:
address_verifications = pl.read_parquet(f"{DATA_PATH}/raw/farcaster-profile_with_addresses-0-1730134800.parquet")
single_address_with_ens = address_verifications.filter(pl.col("fname").str.ends_with(".eth") & ~pl.col("verified_addresses").str.contains(",") & pl.col("verified_addresses").str.contains("0x") )
single_address_with_ens
# Create a column for single address
single_address_with_ens = single_address_with_ens.with_columns(pl.col("verified_addresses").str.slice(2,42).alias("address"))
# Remove users with duplicated address
duplicated_addresses = single_address_with_ens[["address"]].filter(pl.col("address").is_duplicated())["address"].to_list()
single_address_with_ens = single_address_with_ens.filter(~pl.col("address").is_in( duplicated_addresses ))
single_address_with_ens

with open(f"{DATA_PATH}/interim/ens_names.json","r") as f:
  ens_names = json.load(f)

# Add the column 'ens_name'
single_address_with_ens = single_address_with_ens.with_columns(pl.col("address").replace(ens_names,default=None).alias("ens_name"))

# Removing null ENS names
single_address_with_ens = single_address_with_ens.filter(~pl.col("ens_name").is_null())

# Removing fnames different than ENS names
single_address_with_ens = single_address_with_ens.filter( pl.col("fname") == pl.col("ens_name") )

single_address_with_ens

length before removing null ENS names: 4785
length after removing null ENS names: 4463
length before removing fnames different than ENS names: 4463
length after removing fnames different than ENS names: 4250


fname,display_name,avatar_url,bio,verified_addresses,updated_at,fid,address,ens_name
str,str,str,str,str,datetime[ns],i64,str,str
"""cdixon.eth""","""Chris Dixon""","""https://i.imgur.com/ju8z8lD.jpg""","""Crypto/web3. Author of Read Write Own readwriteown.com""","""[""0xe11bfcbdd43745d4aa6f4f18e24ad24f4623af04""]""",2024-06-07 16:44:09.046,25,"""0xe11bfcbdd43745d4aa6f4f18e24ad24f4623af04""","""cdixon.eth"""
"""balajis.eth""","""balajis""","""https://github.com/balajis.png""","""Author of thenetworkstate.com.""","""[""0x0916c04994849c676ab2667ce5bbdf7ccc94310a""]""",2024-06-07 16:44:11.542,37,"""0x0916c04994849c676ab2667ce5bbdf7ccc94310a""","""balajis.eth"""
"""nickgrossman.eth""","""Nick Grossman""","""https://i.imgur.com/LMQMcab.png""","""Partner @USV. Student of cities and the internet. https://nickgrossman.xyz""","""[""0xd3dd62e5ad098d2220e3921d41494fb9754c1bf1""]""",2024-06-07 16:45:06.227,168,"""0xd3dd62e5ad098d2220e3921d41494fb9754c1bf1""","""nickgrossman.eth"""
"""dfmerin.eth""","""Dave Merin 🦊🎩""","""https://lh3.googleusercontent.com/kJL6ipDlIoSit5uJuCk9DAn8_2RBkfOR_6d5Hi5yD37PVFWAWBbYxKV69NbfRcWifIUjUms5FvKzxLeR6-CooOONgBG4vQsWyq0b""","""Head of Corp Dev @ ConsenSys // dfmerin.com // dfmerin.twitter ""","""[""0x282173335ae1f328b34eb9acadc052e59e70a89f""]""",2024-06-07 16:45:24.742,262,"""0x282173335ae1f328b34eb9acadc052e59e70a89f""","""dfmerin.eth"""
"""clacla.eth""","""clacla""","""https://i.imgur.com/vLA4EnP.jpg""","""Builder""","""[""0xd06d9c4e5fe6ee578e4ee5e539ecb8b3453df029""]""",2024-06-07 16:48:06.799,699,"""0xd06d9c4e5fe6ee578e4ee5e539ecb8b3453df029""","""clacla.eth"""
…,…,…,…,…,…,…,…,…
"""corporatefilth.eth""","""Pete""","""https://ipfs.decentralized-content.com/ipfs/bafybeifw7e7taqmtudrre7hwewdwiynlbhr232l5wf3xtrhexvj22nswrq/images/5255.png""","""""","""[""0xc9706e271241d24dcb1b41119b4096b8116c662a""]""",2024-10-28 13:09:00.099902,1184,"""0xc9706e271241d24dcb1b41119b4096b8116c662a""","""corporatefilth.eth"""
"""mrgor.eth""","""MrGor.ETH""","""https://imagedelivery.net/BXluQx4ige9GuW0Ia56BHw/f778408b-8c48-4cb6-1675-6060ba9a9200/original""","""Software Engineer | Backend Developer | Smart Contract Developer | Web3 & DeFi Enthusiast""","""[""0x6ee02cc6acfc6af5912499ee265b92b195b49e44""]""",2024-10-28 14:22:23.440168,871086,"""0x6ee02cc6acfc6af5912499ee265b92b195b49e44""","""mrgor.eth"""
"""sio.eth""","""sio""","""https://imagedelivery.net/BXluQx4ige9GuW0Ia56BHw/9f9bffa0-c311-4571-7a19-9e8439cc6400/original""","""Lead Product Designer at aragon.org ⟡ Cyclist ⟡ Music Nerd ⟡ Sourdough Dad""","""[""0xd5fb864acfd6bb2f72939f122e89ff7f475924f5""]""",2024-10-28 14:50:10.912488,6116,"""0xd5fb864acfd6bb2f72939f122e89ff7f475924f5""","""sio.eth"""


In [12]:
df = single_address_with_ens.sample(len(single_address_with_ens)-1,seed=1)
df

fname,display_name,avatar_url,bio,verified_addresses,updated_at,fid,address,ens_name
str,str,str,str,str,datetime[ns],i64,str,str
"""shaun3.eth""","""Shaun3""","""https://i.imgur.com/sERoq1l.jpg""","""Digitalist | Squig #8654 | http://oncyber.io/shaun3 ""","""[""0xb05dec2370a17478e726452bbfcd093f64f5b900""]""",2024-06-07 17:23:16.835,13820,"""0xb05dec2370a17478e726452bbfcd093f64f5b900""","""shaun3.eth"""
"""ismayle.eth""","""Ismayle""","""https://imagedelivery.net/BXluQx4ige9GuW0Ia56BHw/6e311117-dda9-403c-8bc0-94cd45bbc100/rectcrop3""","""Exploring Web3 🌀 The Future ✨""","""[""0xa6d13c1cf71837b60df5e1dd51a1faa841d01a92""]""",2024-06-23 19:22:00.092,710668,"""0xa6d13c1cf71837b60df5e1dd51a1faa841d01a92""","""ismayle.eth"""
"""djchrisallen.eth""","""Chris Allen""","""https://i.seadn.io/gae/qP9cvTCnAC6e0Hka7t-ucv6LLlEnoypUWcuAn9f1dqUh0wICnyxGXlFlfu1p9glOFCD7eCOnaYVCrdNqiTyXv4kR6Wy65PF-VQoOYQ0?w=500&auto=format""","""Design @ Media.Monks (Formula.Monks) | UX/UI Design & Strategy | Web3 Advocate | Founder | Advisor | Previously Apple, Google, once a player of records.""","""[""0xef1c5987eb809afd075b18515e889117a3606693""]""",2024-10-23 16:46:30.881168,5738,"""0xef1c5987eb809afd075b18515e889117a3606693""","""djchrisallen.eth"""
"""zebez.eth""","""Zebez""","""https://imagedelivery.net/BXluQx4ige9GuW0Ia56BHw/a9982ebc-bf43-45f6-0862-6b94a5b29d00/original""","""In life, I value real relationships, share my passions and always look for inspiration around.""","""[""0x0e9cfbbadbe25f440af3e94cebc1fb6ce8e87daa""]""",2024-06-08 16:49:07.141,534676,"""0x0e9cfbbadbe25f440af3e94cebc1fb6ce8e87daa""","""zebez.eth"""
"""14m4r.eth""","""l4m4r""","""https://i.imgur.com/Kh7yvWm.jpg""","""Working on an MEV bot to make some cash so I can buy myself a Noun 😊""","""[""0x7f31390f9e565cd2bd9c99f49bfd5d14dacabbc5""]""",2024-08-27 02:56:48.698,398235,"""0x7f31390f9e565cd2bd9c99f49bfd5d14dacabbc5""","""14m4r.eth"""
…,…,…,…,…,…,…,…,…
"""didbit.eth""","""Didbit 🎩🐲🐉⛓️""","""https://i.imgur.com/geGC9Fe.png""","""From0tohero 🌿 /onchain follow /guild""","""[""0x984253cad96157b89255f4674e5362fc0feb69ee""]""",2024-10-23 14:59:51.275820,461839,"""0x984253cad96157b89255f4674e5362fc0feb69ee""","""didbit.eth"""
"""mataverse.eth""","""Mataverse""","""https://i.imgur.com/GT4AkR2.jpg""","""Mataverse.eth Mataverse.lens""","""[""0xd174ad0d1e36c7a60cd922e966354095233f8f23""]""",2024-06-07 17:58:51.204,20941,"""0xd174ad0d1e36c7a60cd922e966354095233f8f23""","""mataverse.eth"""
"""airhappy.eth""","""airhappy""","""https://imagedelivery.net/BXluQx4ige9GuW0Ia56BHw/b96fb801-522c-4d75-9f14-2dd3ab798100/original""","""I like WEB 3""","""[""0x64349bc61d02c07e2f6e51be6d2d6bbe9681f32e""]""",2024-06-07 23:18:17.877,311192,"""0x64349bc61d02c07e2f6e51be6d2d6bbe9681f32e""","""airhappy.eth"""


In [13]:
# verified_ens = {str(value): True for value in df[0:120]['fname']}
# verified_ens

In [14]:
verified_ens = {'shaun3.eth': True,
 'ismayle.eth': True,
 'djchrisallen.eth': True,
 'zebez.eth': False,
 '14m4r.eth': True,
 'funnyking.eth': False,
 'ppc.eth': True,
 'zineuc.eth': True,
 'lilicrypto.eth': True,
 'deronsong.eth': True,
 'dackieswap.eth': True,
 'macwisnia.eth': True,
 'coinwang.eth': True,
 'sogito.eth': True,
 'baeshy.eth': True,
 'swixel.eth': True,
 'limone.eth': True,
 'a1lexen.eth': True,
 'yotong.eth': True,
 'mallcop.eth': True,
 'robertu.eth': False,
 'criptogringo.eth': True,
 'db2305.eth': True,
 'yuriy.eth': True,
 '1nvisible.eth': True,
 'waterbeast.eth': True,
 '0xadex.eth': True,
 'tate-ling.eth': True,
 'aplex.eth': True,
 'trouble25.eth': True,
 'rosaknopp.eth': True,
 'evmlion.eth': False,
 'naes12345678910.eth': True,
 'snowsledge.eth': True,
 'minsu89.eth': True,
 'ivanmar.eth': True,
 'foxall.eth': True,
 'javierdv2110.eth': True,
 'franciscocaeiro.eth': True,
 'timtim.eth': True,
 'steve.eth': True,
 'destiner.eth': False,
 'donnaa.eth': True,
 'rahul.eth': True,
 'franiv.eth': True,
 'lidayho.eth': True,
 'dikci.eth': True,
 'cryptomew.eth': True,
 'fafa1.eth': True,
 'asad7.eth': True,
 'madeinathens.eth': True,
 'bartlisa.eth': True,
 'cryptobingoo.eth': True,
 'cryptobabble.eth': True,
 'defilaunchapp.eth': True,
 'dmytrowski.eth': True,
 'incarterseyes.eth': True,
 'btcfounder.eth': True,
 'cameron.eth': True,
 'rtist.eth': True,
 'x0777x.eth': True,
 '0xsonicad.eth': True,
 'jieun.eth': True,
 'galxee.eth': True,
 'reyem.eth': True,
 'ebatplotno.eth': True,
 'kmburnside.eth': True,
 'koko418.eth': True,
 'abuhafsa.eth': True,
 'fliguy.eth': False,
 'alex89.eth': True,
 'mikeneuder.eth': True,
 'taz643.eth': True,
 'matars.eth': True,
 'nimaleophotos.eth': True,
 'ihateanime.eth': True,
 'marziqasemi.eth': True,
 'blubear.eth': True,
 'wavekeeper.eth': True,
 'zaydo.eth': True,
 'altefide.eth': True,
 'mehari.eth': True,
 'andreachiampo.eth': True,
 'rabool.eth': False,
 'arfii.eth': True,
 'faxme.eth': False,
 'ansenkim.eth': True,
 'leonort.eth': True,
 'rohitsethi.eth': True,
 'tunaan.eth': True,
 'cryptolinx.eth': True,
 'kaval.eth': True,
 'vahidx.eth': True,
 'cordobes.eth': True,
 'onchaindefi.eth': True,
 'chia1.eth': True,
 'basedmc.eth': True,
 'george01.eth': True,
 'cisk6.eth': True,
 '575444.eth': True,
 'phantomk.eth': True,
 '0xkreal.eth': True,
 'zoye993.eth': True,
 'aureliu.eth': True,
 'nonart.eth': True,
 'gaudenzio.eth': True,
 'madlog1c.eth': True,
 'lazybaer.eth': True,
 'nekrasovdanylo.eth': True,
 'uspen.eth': True,
 'juisepppe.eth': True,
 'garzione.eth': True,
 'metajohnn.eth': True,
 'dyslexic.eth': True,
 'deskcripto.eth': True,
 'fishker.eth': True,
 'yeradaoti.eth': False,
 'vinayaks.eth': True,
 'wanderloots.eth': True,
 'pushgor.eth': True
}

In [15]:
list_of_humans2 = [key for key,value in verified_ens.items() if value is False]
list_of_humans2

['zebez.eth',
 'funnyking.eth',
 'robertu.eth',
 'evmlion.eth',
 'destiner.eth',
 'fliguy.eth',
 'rabool.eth',
 'faxme.eth',
 'yeradaoti.eth']

### 3. Filtered users ( removed users who casted bot words and with < 10 casts)

In [40]:
casts = pl.read_parquet(f"{DATA_PATH}/interim/casts_samples.parquet")

df = casts.group_by('fid').agg([
  pl.col("text").count().alias("casts_count"),
  pl.col('text').str.to_lowercase().str.concat(' ').alias('text')
                               ])

bot_words = ["degen", "zora", "moxie", "mint", "memecoin"]
pattern = "|".join(bot_words)
df = df.filter((pl.col("casts_count") > 10) & (~pl.col("text").str.contains(pattern)))

df = df.join(last_fnames,how="left",on="fid",coalesce=True)[["fid","fname"]]
df

fid,fname
i64,str
422617,"""heejae"""
317233,"""ethanolo"""
219313,"""ansacervicalis"""
709056,"""ciervo-plasma-2"""
516071,"""totosi"""
…,…
354669,"""dappykit"""
512361,"""blackwhite"""
490758,"""forcer"""


In [41]:
# result_dict = {str(value): True for value in df['fname']}
# result_dict

In [42]:
filtered_users = {
 'nightguardian': True,
 'artnome': True,
 'daydreamer': True,
 'pastafarian': True,
 'shogan': True,
 'undeadsoldier125': True,
 'giveitaway': True,
 'ciervo-plasma-2': True,
 'crypto-partner': True,
 'heejae': True,
 'kathfishing': True,
 'silvermage': True,
 'discovisco': True,
 'noctis': False,
 'tanoaki': True,
 'bonpon': True,
 'ichronichd': True,
 'diogomonica': False,
 'enderscrysis': True,
 'coffeyyy': True,
 'truongleeth': True,
 'ethanolo': True,
 'kaizerrng': True,
 'joshadumb': True,
 'genial': True,
 'bauerboun243': True,
 'pavlo34': True,
 'coolmodi': True,
 'ansacervicalis': True,
 'totosi': True,
 'xannyboo': True,
 'garrytan': False,
 'kiros': True,
 'erfan92472': True,
 'dgct.eth': True,
 'kunal': False,
 'librarian': True,
 'dezdropped1t': True,
 'imranrafique': True,
 'sara28': True,
 'gopinath-m': True,
 'nemo': True,
 'noun74': False,
 'petrolivkan': True,
 'redrosary': True,
 'resham': True,
 'moonboyyy': True,
 'rojan': True,
 'loaf': True,
 'elise9': True,
 'dinokiller276': True,
 'hellsmate': True,
 'yeetgodml': True,
 'anmworld3': True,
 'fwooji': True,
 'ryansmith': False,
 'jasoncdukes': True,
 'fireworks': True,
 'sheeshroom': True,
 'zachlieberman': True,
 '0xzhino': True,
 'fraggue': True,
 'apricotchan': True,
 'artyyy11': True,
 'shabalin': True,
 'aweissman': True,
 'nogsbot': True,
 'cornpotage': True,
 'cigz': True,
 'texturepapi': True,
 'yupuday': True,
 'ramik': True,
 'surrealserpent': True,
 'mizann': True,
 'chicagobulls': True,
 'mosami': True,
 'positron.eth': True,
 'wowman': True,
 'alibek1993': True,
 'gabrielle115': True,
 '0xcharmed': True,
 'pussycat': True,
 'funnyperson': True,
 'gudrun21': True,
 'thundertoko': True,
 'swatchesbot': True,
 'nightflyer321': True,
 'diskrancher.eth': True,
 'chromaticcrow': True,
 'cryptobazz': True,
 'madvi': False,
 'None': True,
 'fri564': True,
 'quix': True,
 'masha58': True,
 'ryanshea.eth': False,
 'akshaan': False,
 'fernandeseric': True,
 'pris': True,
 'jbsdc': True,
 'missmarci': True,
 'mickeloo': True,
 'austilicious123': True,
 'snaks': True,
 'poolie': True,
 'biancasama': True,
 'dishwasher': True,
 'sfranks.eth': False,
 'day-k': False,
 'minorijp': True,
 'shapiro500': True,
 'chatim': True,
 'sethpate': True,
 'mojtab': True,
 'cobie': False,
 'ilhyun2': True,
 'alphonse86.eth': False,
 'sriramk.eth': False,
 'hamdy': True,
 'coolguystorm': True,
 'helenprest': True,
 'vangogh': True,
 'kacuatro.eth': True,
 'iraj': True,
 'zxc81819': True,
 'sunrogue': True,
 'kodzuken': True,
 'opengunnft': True,
 'gospelofchange': True,
 'bombah': True,
 'hertje73': True,
 'paulp': True,
 'questtowncc': True,
 'unknownrayn119': True,
 'howmy': True,
 'downsandtowns': True,
 'bonemelt': True,
 'suhailkakar': False,
 'iexec': True,
 'infiniteorb': True,
 'rets': True,
 'vt': False,
 'terence.eth': False,
 'glif': True,
 'unlimitmood.eth': True,
 'elenithinks': False,
 'madzane': True,
 'bfortch': True,
 'cqb': False,
 'xngmi': False,
 'darkmythgod': True,
 'micsolana': False,
 'spear49': True,
 'helenzanyy': True,
 'oliviahelen': True,
 'etherspot': True,
 'joksta': True,
 'mely.eth': True,
 'friskris': True,
 'verybig': True,
 'quie': True,
 'digitaldiva': True,
 'cooper12': True,
 'dmva': True,
 'daju': True,
 'jj96': True,
 'bap': False,
 'bookishoxygen': True,
 'fabric': True,
 'pubcrawler': True,
 'scarlet-thread': True,
 'sds.eth': False,
 'dollarbro': True,
 'loomdart': False,
 'dragon22455': True,
 'redboyo': True,
 'cryptotoad': True,
 'gkeptm9999': True,
 'bura7501': True,
 'snowbeastly': True,
 'sanidul': True,
 'hanauien': True,
 'udiverse': False,
 'wildlover': True,
 'quixado.eth': False,
 'koderholic': True,
 'otosmundos': True,
 'gzilla': True,
 'thejayess': True,
 'zvilas': True,
 'zheltukheen': True,
 'phosphorizer': True,
 'dronuu': True,
 'pepelady': True,
 'red-panda': True,
 'dyguio': True,
 'biohackerdao': True,
 'rainbow94': True,
 'tongxxxxiaolei': True,
 'chrisu': True,
 'brovsky': True,
 'defimaven': True,
 'sunyuping': True,
 'camarilla': False,
 'ohaijens': True,
 'erik': False,
 'worldcoin': False,
 'goatmilk': True,
 'zanzoun': True,
 'stoicism': True,
 'kiln': False,
 'ec3': True,
 'ytjr': True,
 'dostesina': True,
 'oyhaveasean': True,
 'temoha62rus': True,
 'simple9': True,
 'leuts.eth': True,
 'cr1veth': True,
 'christmastree': True,
 'dappykit': True,
 'cheeseb0wl': True,
 'navie': True,
 'forcer': True,
 'henryzz': True,
 'andreicretu124': True,
 'iredranger': True,
 'jfz': True,
 'eve633': True,
 'd3v0nt3': False,
 'fokyz': True,
 'antidotpng': True,
 'blackwhite': True,
 'carlowes': True,
 'traderokaku': True,
 'alleeee': True,
 'c9grandparent': True,
 'stuz0r': True,
 '9s0z': True,
 'soundsonic1': True,
 'toly': False,
 'cruxiall': True,
 'sarah-n': True,
 'rainehhh': True
}


In [43]:
list_of_humans3 = [key for key,value in filtered_users.items() if value is False]
list_of_humans3

['noctis',
 'diogomonica',
 'garrytan',
 'kunal',
 'noun74',
 'ryansmith',
 'madvi',
 'ryanshea.eth',
 'akshaan',
 'sfranks.eth',
 'day-k',
 'cobie',
 'alphonse86.eth',
 'sriramk.eth',
 'suhailkakar',
 'vt',
 'terence.eth',
 'elenithinks',
 'cqb',
 'xngmi',
 'micsolana',
 'bap',
 'sds.eth',
 'loomdart',
 'udiverse',
 'quixado.eth',
 'camarilla',
 'erik',
 'worldcoin',
 'kiln',
 'd3v0nt3',
 'toly']

### Now, let's use the list humans do far to expand it by taking a look af followed accounts

In [44]:
list_of_humans4 = list(set(list_of_humans1 + list_of_humans2 + list_of_humans3))
len(list_of_humans4)

42

In [46]:
# Get a sample of users followed by our current list of fids 
human_fids4 = last_fnames.filter(pl.col("fname").is_in(list_of_humans4))["fid"].to_list()
links_lazy = pl.scan_parquet(f"{DATA_PATH}/raw/farcaster-links-0-1730134800.parquet")
links_lazy = links_lazy.select(["fid","target_fid"])
links_lazy = links_lazy.filter(pl.col("fid").is_in(human_fids4))
# Limit each user to have 50 following ones on the list
links_lazy = links_lazy.group_by("fid").head(50).sort("fid")
links = links_lazy.collect()
links

fid,target_fid
i64,i64
17,2
17,13
17,11
17,20
17,3
…,…
658756,4804
658756,602
658756,3621


In [47]:
df = links.drop("fid").rename({"target_fid":"fid"})
df = df.filter(~df["fid"].is_duplicated()).join(last_fnames,on="fid",coalesce=True)
df

fid,fname
i64,str
4466,"""joncharbonneau"""
13901,"""marissaposner"""
11299,"""tempetechie.eth"""
4996,"""myandroid.eth"""
5779,"""pifafu"""
…,…
128,"""omeze"""
5356,"""s12chung"""
2084,"""dob.eth"""


In [48]:
# followings = {str(value): True for value in df['fname']}
# followings = {f"https://warpcast.com/{key}":True for key in followings.keys()}
# followings

In [49]:
followings = {'https://warpcast.com/willhennessy': True,
 'https://warpcast.com/bitcery': True,
 'https://warpcast.com/defiprime': True,
 'https://warpcast.com/princesspancakes': False,
 'https://warpcast.com/vmark': True,
 'https://warpcast.com/meltem': False,
 'https://warpcast.com/czar': False,
 'https://warpcast.com/daojoan.eth': False,
 'https://warpcast.com/weboftrust': False,
 'https://warpcast.com/bren': False,
 'https://warpcast.com/carlbeek': False,
 'https://warpcast.com/chax': True,
 'https://warpcast.com/adityapk': False,
 'https://warpcast.com/kylesamani': False,
 'https://warpcast.com/katy': True,
 'https://warpcast.com/haurog.eth': True,
 'https://warpcast.com/juanx': False,
 'https://warpcast.com/pintail': False,
 'https://warpcast.com/fuzzcheek': True,
 'https://warpcast.com/joncharbonneau': False,
 'https://warpcast.com/thebower.eth': False,
 'https://warpcast.com/kangster': True,
 'https://warpcast.com/northchop': False,
 'https://warpcast.com/myfriendkappy.eth': False,
 'https://warpcast.com/jd': False,
 'https://warpcast.com/tikhon': False,
 'https://warpcast.com/ajwaxman': False,
 'https://warpcast.com/myandroid.eth': False,
 'https://warpcast.com/richwidmann': False,
 'https://warpcast.com/0xzak': False,
 'https://warpcast.com/ncitron.eth': False,
 'https://warpcast.com/tempetechie.eth': True,
 'https://warpcast.com/diogomonica': False,
 'https://warpcast.com/rileyluna.eth': False,
 'https://warpcast.com/nishant': False,
 'https://warpcast.com/atifkhan31': False,
 'https://warpcast.com/marissaposner': True,
 'https://warpcast.com/0x01': True,
 'https://warpcast.com/avichalp': False,
 'https://warpcast.com/alexatallah': False,
 'https://warpcast.com/thatguyintech.eth': False,
 'https://warpcast.com/chabe': True,
 'https://warpcast.com/leoclark.eth': True,
 'https://warpcast.com/jvaleska.eth': False,
 'https://warpcast.com/vrypan.eth': False,
 'https://warpcast.com/conley': False,
 'https://warpcast.com/saeed-rhm': True,
 'https://warpcast.com/cerv1': False,
 'https://warpcast.com/offchainlabs': False,
 'https://warpcast.com/pifafu': False,
 'https://warpcast.com/dude': False,
 'https://warpcast.com/mohan': True,
 'https://warpcast.com/sarahzorah': False,
 'https://warpcast.com/jtnichol': False,
 'https://warpcast.com/vladimir': True,
 'https://warpcast.com/kunaguero0000': True,
 'https://warpcast.com/juli': False,
 'https://warpcast.com/hasu': False,
 'https://warpcast.com/ridam': True,
 'https://warpcast.com/basedinstitute': False,
 'https://warpcast.com/supercast': True,
 'https://warpcast.com/joshelman': False,
 'https://warpcast.com/mcbain': False,
 'https://warpcast.com/adrianoferia.eth': False,
 'https://warpcast.com/danny': False,
 'https://warpcast.com/pedrosilva.eth': False,
 'https://warpcast.com/donfrdy': True,
 'https://warpcast.com/neilfahey': False,
 'https://warpcast.com/0xjack': False,
 'https://warpcast.com/omnus': False,
 'https://warpcast.com/montana': False,
 'https://warpcast.com/apoorvlathey': False,
 'https://warpcast.com/morganjweaver': False,
 'https://warpcast.com/amasad': False,
 'https://warpcast.com/0xsmallbrain': False,
 'https://warpcast.com/zachxbt': False,
 'https://warpcast.com/tomu.eth': False,
 'https://warpcast.com/msilb.eth': False,
 'https://warpcast.com/mh1': True,
 'https://warpcast.com/sasquatch': False,
 'https://warpcast.com/dk': False,
 'https://warpcast.com/abranti': False,
 'https://warpcast.com/mihai': False,
 'https://warpcast.com/timdaub.eth': True,
 'https://warpcast.com/trustlessstate': False,
 'https://warpcast.com/superphiz.eth': False,
 'https://warpcast.com/anatcrypto.eth': False,
 'https://warpcast.com/peterk': False,
 'https://warpcast.com/lomelinandrea': False,
 'https://warpcast.com/scottshapiro': False,
 'https://warpcast.com/rue': False,
 'https://warpcast.com/furqan': False,
 'https://warpcast.com/eds': False,
 'https://warpcast.com/harper.eth': False,
 'https://warpcast.com/halal': False,
 'https://warpcast.com/towns': False,
 'https://warpcast.com/blau.eth': False,
 'https://warpcast.com/kryptoshrimp': False,
 'https://warpcast.com/wholesomecrypto.eth': False,
 'https://warpcast.com/clun.eth': False,
 'https://warpcast.com/ev': False,
 'https://warpcast.com/pieratt': False,
 'https://warpcast.com/brexton': False,
 'https://warpcast.com/tusz': False,
 'https://warpcast.com/yoddha.eth': True,
 'https://warpcast.com/toro': False,
 'https://warpcast.com/dschnurr': False,
 'https://warpcast.com/dhof': False,
 'https://warpcast.com/0xpotus': True,
 'https://warpcast.com/garrett': False,
 'https://warpcast.com/mamaxargs.eth': True,
 'https://warpcast.com/prettyjpgmaxi.eth': False,
 'https://warpcast.com/divyav.eth': True,
 'https://warpcast.com/j-m': False,
 'https://warpcast.com/syedshah.eth': False,
 'https://warpcast.com/caspar': False,
 'https://warpcast.com/evanvanness': False,
 'https://warpcast.com/blasteasy': True,
 'https://warpcast.com/atassi': False,
 'https://warpcast.com/sio.eth': True,
 'https://warpcast.com/anaroth': True,
 'https://warpcast.com/qt': True,
 'https://warpcast.com/jconorgrogan': True,
 'https://warpcast.com/evangreenberg': True,
 'https://warpcast.com/denis': False,
 'https://warpcast.com/stelladore': False,
 'https://warpcast.com/yancey': False,
 'https://warpcast.com/qdau.eth': False,
 'https://warpcast.com/ryansmith': False,
 'https://warpcast.com/food0xlens': True,
 'https://warpcast.com/letsglitchit': True,
 'https://warpcast.com/j3': True,
 'https://warpcast.com/garrytan': False,
 'https://warpcast.com/chrismessina': False,
 'https://warpcast.com/manuelmaccou.eth': False,
 'https://warpcast.com/mbaril010': False,
 'https://warpcast.com/0xplutoo': False,
 'https://warpcast.com/evnhsn': True,
 'https://warpcast.com/lauren': False,
 'https://warpcast.com/eriksmith.eth': False,
 'https://warpcast.com/hoolie': True,
 'https://warpcast.com/layer3xyz': False,
 'https://warpcast.com/mikeneuder.eth': False,
 'https://warpcast.com/karmabot': True,
 'https://warpcast.com/dabit3': False,
 'https://warpcast.com/francos.eth': False,
 'https://warpcast.com/miguelito.eth': False,
 'https://warpcast.com/icunucmi': False,
 'https://warpcast.com/loracle.eth': False,
 'https://warpcast.com/nadia': False,
 'https://warpcast.com/yup': False,
 'https://warpcast.com/whaledropair': True,
 'https://warpcast.com/soispoke': False,
 'https://warpcast.com/drakula': True,
 'https://warpcast.com/jseam': False,
 'https://warpcast.com/pzakin': False,
 'https://warpcast.com/rudolf': False,
 'https://warpcast.com/ahn.eth': False,
 'https://warpcast.com/archipelago': True,
 'https://warpcast.com/zain': False,
 'https://warpcast.com/visakanv': False,
 'https://warpcast.com/katsnowstorm': False,
 'https://warpcast.com/resiliencia': False,
 'https://warpcast.com/ai': False,
 'https://warpcast.com/jxom': False,
 'https://warpcast.com/jimpo': False,
 'https://warpcast.com/jchakra.eth': False,
 'https://warpcast.com/lucas': True,
 'https://warpcast.com/kasi.eth': True,
 'https://warpcast.com/strobie.eth': True,
 'https://warpcast.com/0xquick': True,
 'https://warpcast.com/mbar': False,
 'https://warpcast.com/tyson': True,
 'https://warpcast.com/karma': False,
 'https://warpcast.com/leewardbound': True,
 'https://warpcast.com/izzyp': False,
 'https://warpcast.com/cryptocrypto': True,
 'https://warpcast.com/blockdev': False,
 'https://warpcast.com/kmacb.eth': True,
 'https://warpcast.com/goncalo': True,
 'https://warpcast.com/samcm': True,
 'https://warpcast.com/reka': False,
 'https://warpcast.com/tomiiide.eth': True,
 'https://warpcast.com/colinlieb': True,
 'https://warpcast.com/bountycaster': True,
 'https://warpcast.com/toadyhawk.eth': True,
 'https://warpcast.com/raz': False,
 'https://warpcast.com/joey': False,
 'https://warpcast.com/web3auth': True,
 'https://warpcast.com/bbroad.eth': True,
 'https://warpcast.com/martin': False,
 'https://warpcast.com/lightclient': True,
 'https://warpcast.com/wslyvh.eth': False,
 'https://warpcast.com/rubinovitz.eth': False,
 'https://warpcast.com/whitney': True,
 'https://warpcast.com/jacek': False,
 'https://warpcast.com/hypercatcher': False,
 'https://warpcast.com/dcinvestor': False,
 'https://warpcast.com/raimanoj.eth': False,
 'https://warpcast.com/sofito': True,
 'https://warpcast.com/futurefae': False,
 'https://warpcast.com/gunboats': False,
 'https://warpcast.com/mags': False,
 'https://warpcast.com/alexlmiller': False,
 'https://warpcast.com/limone.eth': False,
 'https://warpcast.com/mocareward.eth': True,
 'https://warpcast.com/cxkoda': False,
 'https://warpcast.com/anett': False,
 'https://warpcast.com/viktorbunin': False,
 'https://warpcast.com/matallo.eth': False,
 'https://warpcast.com/mccoy': False,
 'https://warpcast.com/zosegal': False,
 'https://warpcast.com/plumferno.eth': False,
 'https://warpcast.com/nounderline.eth': False,
 'https://warpcast.com/rz': False,
 'https://warpcast.com/benjaminion': True,
 'https://warpcast.com/arjantupan': True,
 'https://warpcast.com/cafeconcriptos': True,
 'https://warpcast.com/ilemi.eth': False,
 'https://warpcast.com/anderselowsson': False,
 'https://warpcast.com/farcaster.eth': False,
 'https://warpcast.com/jasoncrawford.eth': False,
 'https://warpcast.com/yashkarthik.eth': False,
 'https://warpcast.com/sinahab': False,
 'https://warpcast.com/elistark': False,
 'https://warpcast.com/becker': False,
 'https://warpcast.com/wake.eth': False,
 'https://warpcast.com/jamiep.eth': True,
 'https://warpcast.com/tokyy': False,
 'https://warpcast.com/michaelashcroft': True,
 'https://warpcast.com/seanrose': True,
 'https://warpcast.com/thethriller': False,
 'https://warpcast.com/sqx': False,
 'https://warpcast.com/bigshotklim': False,
 'https://warpcast.com/bgrill.eth': False,
 'https://warpcast.com/joeblau': False,
 'https://warpcast.com/hart': False,
 'https://warpcast.com/manggo': False,
 'https://warpcast.com/tieshun': False,
 'https://warpcast.com/pipe': False,
 'https://warpcast.com/mickymalka': False,
 'https://warpcast.com/haydenzadams': False,
 'https://warpcast.com/ramana': False,
 'https://warpcast.com/vinayvasanji.eth': False,
 'https://warpcast.com/literature': False,
 'https://warpcast.com/um': True,
 'https://warpcast.com/pmillerd': False,
 'https://warpcast.com/dankantor': False,
 'https://warpcast.com/minimalgravitas': False,
 'https://warpcast.com/ludo': False,
 'https://warpcast.com/sam-oracle': False,
 'https://warpcast.com/akva': True,
 'https://warpcast.com/boysclub': False,
 'https://warpcast.com/md': False,
 'https://warpcast.com/0xluo.eth': False,
 'https://warpcast.com/abram': False,
 'https://warpcast.com/rnr': False,
 'https://warpcast.com/antoniomendes.eth': False,
 'https://warpcast.com/povwassie': True,
 'https://warpcast.com/luciano': False,
 'https://warpcast.com/yoony12': True,
 'https://warpcast.com/armxy': True,
 'https://warpcast.com/shazow.eth': False,
 'https://warpcast.com/haosu': True,
 'https://warpcast.com/thepencil.eth': True,
 'https://warpcast.com/kody.eth': False,
 'https://warpcast.com/artir': False,
 'https://warpcast.com/we': False,
 'https://warpcast.com/mich-tom': True,
 'https://warpcast.com/suchdao.eth': False,
 'https://warpcast.com/dakh.eth': False,
 'https://warpcast.com/cate': False,
 'https://warpcast.com/mantej.eth': False,
 'https://warpcast.com/tcp': False,
 'https://warpcast.com/hm': False,
 'https://warpcast.com/cybershakti': True,
 'https://warpcast.com/xboxue': False,
 'https://warpcast.com/houseofdrey': True,
 'https://warpcast.com/chainagent.eth': False,
 'https://warpcast.com/fwiz': False,
 'https://warpcast.com/vassilis.eth': False,
 'https://warpcast.com/nickgrossman.eth': False,
 'https://warpcast.com/kevinsimler': False,
 'https://warpcast.com/brianwatroba': False,
 'https://warpcast.com/neilg': False,
 'https://warpcast.com/mem.eth': False,
 'https://warpcast.com/rajgokal': False,
 'https://warpcast.com/alexo': True,
 'https://warpcast.com/gigamesh.eth': False,
 'https://warpcast.com/saxenasaheb.eth': False,
 'https://warpcast.com/ayushm.eth': False,
 'https://warpcast.com/framos.eth': True,
 'https://warpcast.com/rossneilson': False,
 'https://warpcast.com/jahangir': True,
 'https://warpcast.com/rayzhu.eth': True,
 'https://warpcast.com/leosn.eth': True,
 'https://warpcast.com/zachdavidson.eth': False,
 'https://warpcast.com/capn': False,
 'https://warpcast.com/greyseymour.eth': True,
 'https://warpcast.com/emmabwaldron': False,
 'https://warpcast.com/farquest': False,
 'https://warpcast.com/brxckinridge': False,
 'https://warpcast.com/fakepixels': False,
 'https://warpcast.com/iamnomad': False,
 'https://warpcast.com/davidcann.eth': False,
 'https://warpcast.com/danicaswanson': False,
 'https://warpcast.com/limes.eth': False,
 'https://warpcast.com/koloz': False,
 'https://warpcast.com/saemi': False,
 'https://warpcast.com/degenoccultist': True,
 'https://warpcast.com/warren': False,
 'https://warpcast.com/maksim': False,
 'https://warpcast.com/zezima.eth': False,
 'https://warpcast.com/kadar.eth': True,
 'https://warpcast.com/jay': False,
 'https://warpcast.com/xuanzhenz': True,
 'https://warpcast.com/ericxtang': True,
 'https://warpcast.com/arra': True,
 'https://warpcast.com/links': False,
 'https://warpcast.com/quincredible': True,
 'https://warpcast.com/nounsamigos.eth': True,
 'https://warpcast.com/barackobama': True,
 'https://warpcast.com/ralexstokes': False,
 'https://warpcast.com/adamsoffer.eth': False,
 'https://warpcast.com/polar': False,
 'https://warpcast.com/jonathanking.eth': False,
 'https://warpcast.com/toniw.eth': False,
 'https://warpcast.com/kacy': False,
 'https://warpcast.com/shortcake': True,
 'https://warpcast.com/amanda': True,
 'https://warpcast.com/micsolana': False,
 'https://warpcast.com/dennison': False,
 'https://warpcast.com/biron.eth': True,
 'https://warpcast.com/0x2d': False,
 'https://warpcast.com/thenounishprof.eth': False,
 'https://warpcast.com/jonassft': False,
 'https://warpcast.com/koeppelmann.eth': False,
 'https://warpcast.com/vogs117': True,
 'https://warpcast.com/angel': False,
 'https://warpcast.com/dankenney.eth': True,
 'https://warpcast.com/clayallsopp': False,
 'https://warpcast.com/chagitr.eth': True,
 'https://warpcast.com/c': False,
 'https://warpcast.com/cryptogirls': True,
 'https://warpcast.com/zconn.eth': True,
 'https://warpcast.com/veltoss': True,
 'https://warpcast.com/shanvasion': True,
 'https://warpcast.com/dberg': False,
 'https://warpcast.com/max-melmed': False,
 'https://warpcast.com/g': False,
 'https://warpcast.com/kia.eth': True,
 'https://warpcast.com/kalelabs.eth': True,
 'https://warpcast.com/evalir': False,
 'https://warpcast.com/nic': False,
 'https://warpcast.com/sheantu': True,
 'https://warpcast.com/commstark': True,
 'https://warpcast.com/hamuenjoyer': True,
 'https://warpcast.com/salvino': True,
 'https://warpcast.com/srinitude.eth': False,
 'https://warpcast.com/emmanuel': False,
 'https://warpcast.com/raphael': False,
 'https://warpcast.com/scott': True,
 'https://warpcast.com/dapplion': False,
 'https://warpcast.com/ambrosius': False,
 'https://warpcast.com/abhi': False,
 'https://warpcast.com/toby': False,
 'https://warpcast.com/cat': False,
 'https://warpcast.com/chaejonghyun': True,
 'https://warpcast.com/scottbelsky': True,
 'https://warpcast.com/oxb': True,
 'https://warpcast.com/slowcrypto': False,
 'https://warpcast.com/jake': False,
 'https://warpcast.com/chinmay.eth': False,
 'https://warpcast.com/kabirs.eth': False,
 'https://warpcast.com/mostafaalavi1234': True,
 'https://warpcast.com/matt': False,
 'https://warpcast.com/zefram': False,
 'https://warpcast.com/taichi': True,
 'https://warpcast.com/alexphan': True,
 'https://warpcast.com/davidtphung.eth': False,
 'https://warpcast.com/purple': False,
 'https://warpcast.com/sohey.eth': True,
 'https://warpcast.com/jrcasas': True,
 'https://warpcast.com/pfrazee': True,
 'https://warpcast.com/codyb.eth': False,
 'https://warpcast.com/worm.eth': False,
 'https://warpcast.com/quasark': False,
 'https://warpcast.com/consecution.eth': False,
 'https://warpcast.com/a': False,
 'https://warpcast.com/pixel': True,
 'https://warpcast.com/bpo': True,
 'https://warpcast.com/aflock': False,
 'https://warpcast.com/jpetrich': False,
 'https://warpcast.com/gmoney.eth': False,
 'https://warpcast.com/payton': False,
 'https://warpcast.com/sheldon': False,
 'https://warpcast.com/adriamasnou.eth': True,
 'https://warpcast.com/fyi': False,
 'https://warpcast.com/henryl.eth': False,
 'https://warpcast.com/nir.eth': False,
 'https://warpcast.com/musnit.eth': False,
 'https://warpcast.com/path.eth': True,
 'https://warpcast.com/doris': False,
 'https://warpcast.com/grin': False,
 'https://warpcast.com/johanna': False,
 'https://warpcast.com/0xhammer': True,
 'https://warpcast.com/lbeder': True,
 'https://warpcast.com/m1guelpf': False,
 'https://warpcast.com/economist': True,
 'https://warpcast.com/chatgpt': False,
 'https://warpcast.com/coralorca': False,
 'https://warpcast.com/stammy': False,
 'https://warpcast.com/semui.eth': True,
 'https://warpcast.com/mv': False,
 'https://warpcast.com/mathew': False,
 'https://warpcast.com/el4d': False,
 'https://warpcast.com/cynthia': True,
 'https://warpcast.com/hish': True,
 'https://warpcast.com/gwart': False,
 'https://warpcast.com/pentacle.eth': True,
 'https://warpcast.com/jack': True,
 'https://warpcast.com/eguegu': True,
 'https://warpcast.com/tommy': True,
 'https://warpcast.com/m-j-r.eth': True,
 'https://warpcast.com/smc90': False,
 'https://warpcast.com/kneelingbus': False,
 'https://warpcast.com/haunventures': False,
 'https://warpcast.com/moz': False,
 'https://warpcast.com/pseudo': True,
 'https://warpcast.com/jbfromsd': True,
 'https://warpcast.com/chazschmidt': True,
 'https://warpcast.com/ctrlaltvictoria': True,
 'https://warpcast.com/anicorea85': True,
 'https://warpcast.com/cryptobyrde.eth': True,
 'https://warpcast.com/jing': False,
 'https://warpcast.com/z0r0z': False,
 'https://warpcast.com/chatxbt': True,
 'https://warpcast.com/joel': False,
 'https://warpcast.com/kogang': True,
 'https://warpcast.com/craabit': True,
 'https://warpcast.com/211.eth': True,
 'https://warpcast.com/uxhelp.eth': False,
 'https://warpcast.com/sexyboy': True,
 'https://warpcast.com/origami': True,
 'https://warpcast.com/casi': True,
 'https://warpcast.com/entropybender.eth': False,
 'https://warpcast.com/graham': False,
 'https://warpcast.com/dappadan.eth': True,
 'https://warpcast.com/ohyoublockhead': False,
 'https://warpcast.com/boldleonidas': False,
 'https://warpcast.com/bradq': False,
 'https://warpcast.com/parteek': True,
 'https://warpcast.com/coinbasewallet': False,
 'https://warpcast.com/pedropregueiro': False,
 'https://warpcast.com/davidecrapis': True,
 'https://warpcast.com/malone': False,
 'https://warpcast.com/ricmoo': True,
 'https://warpcast.com/qwow': False,
 'https://warpcast.com/jess': False,
 'https://warpcast.com/rrhoover': False,
 'https://warpcast.com/protolambda.eth': False,
 'https://warpcast.com/titey': False,
 'https://warpcast.com/jklb.eth': True,
 'https://warpcast.com/former-elonmusk': True,
 'https://warpcast.com/aviationdoctor.eth': False,
 'https://warpcast.com/benbodhi': True,
 'https://warpcast.com/0xrecruiter': True,
 'https://warpcast.com/srijan.eth': False,
 'https://warpcast.com/pinstripedgator': True,
 'https://warpcast.com/tomx': True,
 'https://warpcast.com/macedo.eth': True,
 'https://warpcast.com/fredriksvantes': False,
 'https://warpcast.com/benroy': False,
 'https://warpcast.com/kirmayer': True,
 'https://warpcast.com/bohm': False,
 'https://warpcast.com/sreeramkannan': False,
 'https://warpcast.com/yb': True,
 'https://warpcast.com/brittkim.eth': False,
 'https://warpcast.com/mene': False,
 'https://warpcast.com/ir': False,
 'https://warpcast.com/nloureiro.eth': True,
 'https://warpcast.com/silentrunner.eth': False,
 'https://warpcast.com/cush.eth': False,
 'https://warpcast.com/aaina': False,
 'https://warpcast.com/yateagle': False,
 'https://warpcast.com/slow': True,
 'https://warpcast.com/cryptohayes': True,
 'https://warpcast.com/alexgrover.eth': False,
 'https://warpcast.com/derek': False,
 'https://warpcast.com/protonogis': True,
 'https://warpcast.com/indexsupply': False,
 'https://warpcast.com/4156': False,
 'https://warpcast.com/nofuckingthanks.eth': True,
 'https://warpcast.com/logristhebard.eth': False,
 'https://warpcast.com/silven': True,
 'https://warpcast.com/jw': True,
 'https://warpcast.com/adrian': True,
 'https://warpcast.com/yaldafasih': True,
 'https://warpcast.com/danger': False,
 'https://warpcast.com/asif18': True,
 'https://warpcast.com/bmann.eth': False,
 'https://warpcast.com/robrecht.eth': False,
 'https://warpcast.com/thebenmeadows.eth': True,
 'https://warpcast.com/m0nt0y4': True,
 'https://warpcast.com/donnie': True,
 'https://warpcast.com/eleon.eth': True,
 'https://warpcast.com/basegod': True,
 'https://warpcast.com/indexcard.eth': True,
 'https://warpcast.com/chriscocreated.eth': True,
 'https://warpcast.com/protocolguild': False,
 'https://warpcast.com/vibesmaxi.eth': False,
 'https://warpcast.com/murena.eth': False,
 'https://warpcast.com/tbenr': True,
 'https://warpcast.com/danco': True,
 'https://warpcast.com/droo4you': True,
 'https://warpcast.com/gami.eth': True,
 'https://warpcast.com/molo': False,
 'https://warpcast.com/pinatacloud': False,
 'https://warpcast.com/faust': True,
 'https://warpcast.com/barnabe': False,
 'https://warpcast.com/malloc': False,
 'https://warpcast.com/bo': True,
 'https://warpcast.com/pc': False,
 'https://warpcast.com/navill': True,
 'https://warpcast.com/swiss': True,
 'https://warpcast.com/perl': True,
 'https://warpcast.com/mattlee': False,
 'https://warpcast.com/htd': False,
 'https://warpcast.com/robknight': True,
 'https://warpcast.com/byungari': True,
 'https://warpcast.com/sorenwrenn': False,
 'https://warpcast.com/akim': False,
 'https://warpcast.com/dli': False,
 'https://warpcast.com/kp3556.eth': True,
 'https://warpcast.com/mrdarp': True,
 'https://warpcast.com/jrf': True,
 'https://warpcast.com/dan': False,
 'https://warpcast.com/androidsixteen.eth': False,
 'https://warpcast.com/dog': True,
 'https://warpcast.com/juan': True,
 'https://warpcast.com/warpcast': True,
 'https://warpcast.com/grady': False,
 'https://warpcast.com/event': True,
 'https://warpcast.com/shanemac.eth': False,
 'https://warpcast.com/rush': False,
 'https://warpcast.com/cedoor': False,
 'https://warpcast.com/defillama': True,
 'https://warpcast.com/leonardo': True,
 'https://warpcast.com/smolrefuel': True,
 'https://warpcast.com/avichal': False,
 'https://warpcast.com/emanuelpinho': False,
 'https://warpcast.com/a16zcrypto': False,
 'https://warpcast.com/drewf.eth': False,
 'https://warpcast.com/regynald': True,
 'https://warpcast.com/seangeng': False,
 'https://warpcast.com/modsoussi': False,
 'https://warpcast.com/nibnalin.eth': False,
 'https://warpcast.com/nick.eth': False,
 'https://warpcast.com/dcbuilder': True,
 'https://warpcast.com/klamt-eth': False,
 'https://warpcast.com/naval': False,
 'https://warpcast.com/darylx': False,
 'https://warpcast.com/joseonom': True,
 'https://warpcast.com/omeze': True,
 'https://warpcast.com/wbnns.eth': False,
 'https://warpcast.com/cardno.eth': True,
 'https://warpcast.com/eusoujp.eth': True,
 'https://warpcast.com/jennadixon.eth': True,
 'https://warpcast.com/rwx': False,
 'https://warpcast.com/haardikkk': False,
 'https://warpcast.com/skynet.eth': True,
 'https://warpcast.com/alegw.eth': True,
 'https://warpcast.com/tomcritchlow': True,
 'https://warpcast.com/edatweets': True,
 'https://warpcast.com/eric.eth': False,
 'https://warpcast.com/bfresh': True,
 'https://warpcast.com/0xfoobar': True,
 'https://warpcast.com/altmbr': False,
 'https://warpcast.com/ivy': True,
 'https://warpcast.com/douglass': False,
 'https://warpcast.com/neshama.eth': True,
 'https://warpcast.com/kunani': True,
 'https://warpcast.com/olanetsoft': False,
 'https://warpcast.com/jamesstewart': False,
 'https://warpcast.com/dob.eth': False,
 'https://warpcast.com/zakir71': True,
 'https://warpcast.com/nickbytes.eth': False,
 'https://warpcast.com/frolic': True,
 'https://warpcast.com/tbi': False,
 'https://warpcast.com/alexmasmej.eth': True,
 'https://warpcast.com/alex-gedevani': True,
 'https://warpcast.com/flx': True,
 'https://warpcast.com/octant': True,
 'https://warpcast.com/philcockfield.eth': False,
 'https://warpcast.com/quintus': True,
 'https://warpcast.com/potatospare': True,
 'https://warpcast.com/gramsci.eth': True,
 'https://warpcast.com/preetjdp': True,
 'https://warpcast.com/ansakrghtn': True,
 'https://warpcast.com/tindz': True,
 'https://warpcast.com/fradamt': False,
 'https://warpcast.com/s12chung': False,
 'https://warpcast.com/nouns-dao': True,
 'https://warpcast.com/sdv.eth': False,
 'https://warpcast.com/vpabundance.eth': True}

In [50]:
followings = {key.split(".com/")[-1]:value for key,value in followings.items()}
list_of_humans5 = [key for key,value in followings.items() if value is False]
list_of_humans5 = list(set(list_of_humans4 + list_of_humans5))
len(list_of_humans5)

416

In [37]:
human_fids = last_fnames.filter(pl.col("fname").is_in(list_of_humans5))["fid"].to_list()
human_fids.sort()
human_fids

[1,
 11,
 13,
 16,
 17,
 29,
 33,
 43,
 52,
 55,
 58,
 61,
 63,
 64,
 68,
 70,
 72,
 81,
 87,
 88,
 90,
 97,
 105,
 112,
 113,
 130,
 132,
 133,
 139,
 144,
 159,
 165,
 166,
 168,
 173,
 176,
 183,
 187,
 188,
 189,
 190,
 193,
 195,
 197,
 200,
 201,
 203,
 209,
 212,
 213,
 215,
 218,
 219,
 234,
 237,
 241,
 243,
 247,
 256,
 257,
 280,
 288,
 296,
 311,
 317,
 357,
 363,
 375,
 377,
 381,
 402,
 412,
 417,
 429,
 469,
 521,
 523,
 555,
 578,
 599,
 626,
 629,
 631,
 656,
 675,
 716,
 737,
 746,
 749,
 767,
 838,
 841,
 849,
 852,
 916,
 941,
 964,
 1010,
 1020,
 1082,
 1102,
 1114,
 1129,
 1236,
 1277,
 1315,
 1349,
 1401,
 1495,
 1513,
 1593,
 1600,
 1626,
 1657,
 1659,
 1668,
 1684,
 1786,
 1797,
 1841,
 1863,
 1970,
 2029,
 2042,
 2084,
 2112,
 2139,
 2170,
 2235,
 2338,
 2407,
 2417,
 2502,
 2606,
 2615,
 2669,
 2670,
 2745,
 2782,
 2802,
 2812,
 2904,
 3017,
 3143,
 3175,
 3305,
 3306,
 3312,
 3329,
 3335,
 3409,
 3605,
 3612,
 3635,
 3692,
 3700,
 3707,
 3734,
 3827,
 3842,


In [38]:
with open(f"{DATA_PATH}/interim/human_fids.json","w") as f:
  json.dump(human_fids,f)

### Check how many of them have more than 10 casts

In [57]:
casts_lazy = pl.scan_parquet(f"{DATA_PATH}/raw/farcaster-casts-0-1730134800.parquet")
casts_lazy = casts_lazy.select(["fid", "text", "created_at"])
casts_lazy = (casts_lazy.filter(pl.col("fid").is_in(human_fids)) )
human_casts = casts_lazy.collect()
human_casts

fid,text,created_at
i64,str,datetime[ns]
1,"""testing""",2023-09-05 23:07:28.064999
1,"""test""",2023-09-05 23:07:28.064999
1,"""another test""",2023-09-05 23:07:28.064999
1,"""another testy test""",2023-09-05 23:07:28.064999
11,"""Italy, summer in the north, winter in the south. Get the food and beauty but also year round good weather.""",2023-09-05 23:07:33.966416
…,…,…
7960,"""Really appreciate it! 💪🏾💪🏾""",2024-10-28 16:56:39.876001
193,"""Virtuals.""",2024-10-28 16:57:08.878923
8004,"""love this rendition very much 🤌""",2024-10-28 16:57:22.518647


In [58]:
human_casts.group_by('fid').agg(pl.col("text").count().alias("casts_count")).filter(pl.col("casts_count") >= 10)

fid,casts_count
i64,u32
20228,204
5779,150
3692,75
8521,15
13505,17046
…,…
239308,391
269739,97
2745,8139


### Conclusion

It was possible to achieve a 416 human fids, with 357 samples having more than 10 casts. As most part of these samples were obtained through inspecting followers of some initial accounts, it is not recommended to use this dataset to evaluate SybilSCAR.